In [1]:
import numpy as np
import gurobipy as gp
import pandas as pd
from scipy.optimize import minimize

In [2]:
def create_model(A, sense, b, obj, opt=gp.GRB.MAXIMIZE, ub=None, lb=None, vtype=None, time_limit=3600):
    # creating model
    model = gp.Model()
    
    if vtype is None:
        vtype = ['C'] * A.shape[1]

    # creating variable an setting the constraints
    if ub is None:
        modx = model.addMVar(A.shape[1], vtype=vtype)
    else:
        modx = model.addMVar(A.shape[1], ub=ub, vtype=vtype)
    mod_con = model.addMConstrs(A, modx, sense, b)

    # setting the objective function
    model.setMObjective(None, obj, 0, sense=opt)
 
    # restricting gurobi logs
    model.Params.OutputFlag = 0
    model.setParam('TimeLimit', time_limit)

    # optimizing the function
    model.optimize()
    
    return model

# Q-1

It costs a company usd 12 to purchase an hour of labor and usd 15 to purchase an hour of capital. If L hours of labor and K units of capital are available, then 0.05L2/3K1/3 machines can be produced. Suppose the company has usd 100,000 to purchase labor and capital. What is the maximum number of machines it can produce? Round to the nearest whole number of machines.

In [3]:
# creting optimal function
def get_machine_produced(x):
    labor = x[0]
    capital = x[1]
    
    machine_produced = 0.05*labor**(2/3)*capital**(1/3)
    
    return -machine_produced
    

In [4]:
# creating cost constraint
cons = ({'type': 'ineq', 'fun': lambda x:  100000 - 12*x[0] - 15*x[1]})

In [5]:
# maximizing the machines produced
opt_q1 = minimize(get_machine_produced,[5000,5000],
                  bounds=[(0,10000),(0,10000)], constraints=cons)

In [6]:
opt_q1.x

array([5555.54971306, 2222.22689622])

In [7]:
np.round(-get_machine_produced(opt_q1.x))

205.0

# Q-2

The file homework4stocks.csv  Download homework4stocks.csvcontains historical monthly returns for 27 companies. The first row contains stock names, and the first column contains the dates. For each company, calculate the estimated mean return and the estimated variance of return. Then calculate the estimated covariances between the companies' returns. Find a portfolio that achieves an expected monthly return of at least 1% and minimizes portfolio variance.  Assume no short selling is allowed. What is the portfolio's standard deviation?  Round to 2 decimal places?  Answer in terms of absolute numbers.  If your answer is 8% then you should enter 0.08.

 

We may or may not get to the content for this problem before it's due.  If we don't then don't worry about it.

In [8]:
stocks = pd.read_csv('homework4stocks.csv')
stocks.head()

,Date,AA,AAPL,AXP,BA,CAT,CSCO,DD,DIS,EK,...,KO,MCD,MMM,MO,MRK,MSFT,PG,T,UTX,WMT
0,Jan-2004,-0.101,0.055,0.075,-0.009,-0.055,0.061,-0.043,0.029,0.107,...,-0.030,0.037,-0.070,0.021,0.030,0.010,0.017,-0.011,0.008,0.015
1,Feb-2004,0.101,0.060,0.031,0.043,-0.030,-0.099,0.035,0.105,0.005,...,0.015,0.099,-0.009,0.035,0.010,-0.040,0.014,-0.058,-0.032,0.106
2,Mar-2004,-0.074,0.130,-0.028,-0.053,0.044,0.018,-0.064,-0.058,-0.083,...,0.012,0.010,0.049,-0.043,-0.074,-0.060,0.023,0.022,-0.063,0.004
3,Apr-2004,-0.114,-0.047,-0.056,0.039,-0.012,-0.113,0.017,-0.078,-0.015,...,0.005,-0.047,0.056,0.017,0.064,0.048,0.014,0.028,-0.001,-0.045
4,May-2004,0.023,0.088,0.036,0.078,-0.031,0.070,0.014,0.019,0.025,...,0.015,-0.030,-0.018,-0.133,0.006,0.004,0.019,-0.048,-0.015,-0.020


In [9]:
stock_mean = stocks.iloc[:, 1:].apply('mean', axis=0)
stock_mean

AA     -0.005779
AAPL    0.049286
AXP     0.007494
BA      0.010662
CAT     0.012390
CSCO    0.002532
DD      0.003065
DIS     0.007844
EK     -0.007688
FDX     0.006130
GE     -0.001584
GT      0.018727
HPQ     0.012325
IBM     0.006870
IP      0.003143
JNJ     0.004545
JPM     0.007091
KO      0.003649
MCD     0.016390
MMM     0.003000
MO      0.011961
MRK     0.002844
MSFT    0.004221
PG      0.005662
T       0.004818
UTX     0.007649
WMT     0.001805
dtype: float64

In [10]:
stock_var = stocks.iloc[:, 1:].cov()
stock_var

,AA,AAPL,AXP,BA,CAT,CSCO,DD,DIS,EK,FDX,...,KO,MCD,MMM,MO,MRK,MSFT,PG,T,UTX,WMT
AA,0.013656,0.006025,0.007063,0.004233,0.008318,0.004408,0.005365,0.004893,0.004586,0.004916,...,0.002070,0.001884,0.003601,0.001283,0.002432,0.003552,0.001582,0.002696,0.003637,0.001022
AAPL,0.006025,0.014295,0.006367,0.003505,0.004729,0.003312,0.003641,0.003477,0.002872,0.003340,...,0.001133,0.002767,0.002614,0.002288,0.001295,0.003710,0.001251,0.002266,0.002393,-0.000312
AXP,0.007063,0.006367,0.017277,0.005524,0.008176,0.005145,0.007190,0.005690,0.004734,0.006812,...,0.001420,0.001436,0.004510,0.001823,0.001134,0.004006,0.002385,0.002160,0.004179,0.000773
BA,0.004233,0.003505,0.005524,0.006585,0.003458,0.002636,0.003765,0.003279,0.005510,0.002633,...,0.000982,0.001156,0.002239,0.001244,0.002613,0.001521,0.001063,0.000953,0.003095,0.000043
CAT,0.008318,0.004729,0.008176,0.003458,0.010547,0.004454,0.004997,0.004118,0.003857,0.005635,...,0.002025,0.001793,0.003412,0.000787,0.001137,0.002747,0.002305,0.002742,0.003426,0.000914
CSCO,0.004408,0.003312,0.005145,0.002636,0.004454,0.006239,0.003343,0.003171,0.002940,0.003255,...,0.001462,0.000805,0.002396,0.000569,0.001451,0.002552,0.001373,0.001678,0.002034,0.000421
DD,0.005365,0.003641,0.007190,0.003765,0.004997,0.003343,0.005926,0.003772,0.003683,0.003924,...,0.001206,0.001271,0.002797,0.001567,0.001835,0.001855,0.001279,0.001226,0.002760,0.001033
DIS,0.004893,0.003477,0.005690,0.003279,0.004118,0.003171,0.003772,0.004637,0.002549,0.002931,...,0.001433,0.001527,0.002031,0.001324,0.001623,0.001985,0.001097,0.000952,0.002446,0.000640
EK,0.004586,0.002872,0.004734,0.005510,0.003857,0.002940,0.003683,0.002549,0.022754,0.002976,...,0.000805,0.002389,0.002289,0.001276,0.003146,0.003594,0.001283,0.001283,0.002866,0.001622
FDX,0.004916,0.003340,0.006812,0.002633,0.005635,0.003255,0.003924,0.002931,0.002976,0.006042,...,0.001197,0.001042,0.003122,0.000287,0.000301,0.001662,0.001617,0.001598,0.002240,0.001225


In [11]:
n = stock_var.shape[1]
n

27

Each decision variable will be the weight of the stocks in the porfolio. Overall portfolio variance will the square of weights for the stock multiplied by the stock variance.

There will be two constraint - 
1. Sum of all the weights will be one
2. Expected return should be more than 1% (0.01)

In [12]:
def expected_mean_const(x):
    portfolio_mean = 0
    for i in range(n):
        portfolio_mean += x[i] * stock_mean[i]
    
    return portfolio_mean - 0.01

In [13]:
const1 = ({'type': 'ineq', 'fun': expected_mean_const})

In [14]:
def weight_sum_const(x):
    sum_weights = 0
    for i in range(n):
        sum_weights += x[i]
    
    return sum_weights - 1

In [15]:
const2 = ({'type': 'eq', 'fun': weight_sum_const})

In [16]:
cons = [const1, const2]

In [17]:
def get_portfolio_var(x):
    portfolio_var = 0
    for i in range(n):
        for j in range(n):
            portfolio_var += x[i] * x[j] * stock_var.iloc[i, j]
    
    return portfolio_var

In [18]:
# minimize the portfolio variance
opt_q2 = minimize(get_portfolio_var, [1/n]*n, bounds=[(0,1)]*n, constraints=cons)

In [19]:
opt_q2.x

array([1.13508214e-17, 5.18341614e-02, 1.80037923e-18, 4.35122269e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.92655116e-19,
       2.67578964e-19, 0.00000000e+00, 9.26623890e-19, 1.04100343e-17,
       1.33758459e-03, 1.04083409e-17, 1.44403205e-17, 8.54671156e-02,
       0.00000000e+00, 2.84316581e-02, 1.71317853e-01, 0.00000000e+00,
       2.32483450e-01, 0.00000000e+00, 0.00000000e+00, 9.66903122e-02,
       6.52337194e-02, 0.00000000e+00, 2.66769024e-01])

In [20]:
# mean
expected_mean_const(opt_q2.x) + 0.01

0.010000000000606364

In [21]:
# standard deviation
np.round(opt_q2.fun**0.5, 2)

0.03

# Q-3

The file variable_selection.csv  Download variable_selection.csvcontains observations of variables y, x1, x2, and x3. Here, y is the dependent variable. We want to choose a linear model that uses at most two independent variables such that the sum of squared residuals is minimized. This can be formulated as a constrained quadratic programming problem.

Screen Shot 2021-03-22 at 10.17.11 AM.png  

This is called best subset problem that is usually very hard to solve. We will solve this problem by enumeration. Run six OLS regressions (3 with one independent variable and three more with two variables each) and choose the regression that best fits the data. You can run each regression in R using the lm routine.

In [22]:
var_sel = pd.read_csv('variable_selection.csv')
var_sel

,Unnamed: 0,y,x1,x2,x3
0,1,42.412844,4.529686,7.172525,15.356190
1,2,24.820174,2.516030,4.377053,6.084709
2,3,26.617801,3.038386,4.411439,11.563392
3,4,21.329637,1.602293,4.225374,9.365461
4,5,34.183805,4.148982,5.618860,8.534010
...,...,...,...,...,...
95,96,24.656431,2.607587,4.077544,12.309712
96,97,39.280398,2.874912,7.871697,11.435131
97,98,30.085919,3.768077,4.622765,16.279047
98,99,31.639203,2.645656,5.991489,7.619829


In [23]:
from sklearn.linear_model import LinearRegression

In [24]:
# enumerating different instance
combinations = [['x1'], ['x2'], ['x3'], ['x1', 'x2'], ['x1', 'x3'], ['x2', 'x3']]
sse_list = []

for c in combinations:
    y = var_sel['y']
    x = var_sel[c]
    
    model = LinearRegression()
    model.fit(x, y)
    sse = np.sum((model.predict(x) - y)**2)
    
    sse_list.append(sse)

In [25]:
min_sse_idx = sse_list.index(min(sse_list))
combinations[min_sse_idx]

['x1', 'x2']

# Q-4

The file nflratings.csv  Download nflratings.csvcontains the results of 256 regular-season NFL games from the 2009 season. The teams are indexed 1 to 32 as shown below:

Screen Shot 2020-10-15 at 4.46.40 PM.png

The csv data file contains a matrix with the following columns:

• Week (1-17)
• Home Team Index (1-32 from the table above)

• Visiting Team Index (1-32 from the table above)

• Home Team Score
• Visiting Team Score

For example, the first game in the matrix is team 25 Pittsburgh versus team 31 Tennessee, played at Pittsburgh. Pittsburgh won the game by a score of 13 to 10, and the point spread (home team score minus visitor team score) is 3. A positive point spread means that the home team won; a negative point spread indicates that the visiting team won.

The goal of this problem is to determine a set of ratings for the 32 NFL teams that most accurately predicts the actual outcomes of the games played, similar to homework 1. Here however, we will also incorporate a 'home field advantage' that adds some number of points to the predicted point spread.  Use NLP to find the ratings that best predict the actual point spreads observed. The model will estimate the home team advantage and the ratings.  The model accounts for the home team advantage by adding a constant (which you need to solve for) to the predicted point spread.  The objective is to minimize the sum of squared prediction errors. You will need to calculate the following:

• Actual Point Spread = Home Team Score – Visiting Team Score

• Predicted Spread = Home Team Rating – Visitor Team Rating + Home Team Advantage

• Prediction error = Actual Point Spread – Predicted Point Spread

Your goal is to minimize: Screen Shot 2021-03-22 at 10.22.05 AM.png  

You will also need to normalize the ratings (like you did in HW1). To do this, you set the actual average of the ratings to be 85 (this is somewhat arbitrary but based on the well-known Sagarin rating system). What do these ratings mean: If two teams had ratings of 82 and 91, then the second team would be predicted to win by 9 points if the game was played on a neutral field.

Formulate this as an NLP and solve it.

How many games (of the 256 played) does this model predict the winner correctly?

In [26]:
rating = pd.read_csv('nflratings.csv', names=[
    'week', 'home_team_index', 'visit_team_index', 'home_score', 'visit_score'])
rating['actual_spread'] = rating['home_score'] - rating['visit_score']
rating.head()

,week,home_team_index,visit_team_index,home_score,visit_score,actual_spread
0,1,25,31,13,10,3
1,1,2,17,19,7,12
2,1,29,26,28,0,28
3,1,21,32,23,17,6
4,1,3,16,38,24,14


Decision variables will be the home advantage 

In [27]:
def sum_prediction_error(x):
    error_list, _ = prediction_error(x)
    return sum(error_list)

In [28]:
def prediction_error(x):
    prediction_error_list = []
    prediction_spread_list = []

    for i in range(len(rating)):
        actual_spread = rating.iloc[i, 5]
        
        home_team_idx = rating.iloc[i, 1]
        visit_team_idx = rating.iloc[i, 2]
        predicted_spread = x[home_team_idx-1] - x[visit_team_idx-1] + x[-1]
        
        prediction_spread_list.append(predicted_spread)
        prediction_error_list.append((actual_spread - predicted_spread)**2)
    
    return prediction_error_list, prediction_spread_list
        

In [29]:
# minimize the error
opt_q4 = minimize(sum_prediction_error,[100]*32 + [0], bounds=[(0,100)]*33)

In [30]:
opt_q4.fun**0.5/256

0.8093169299006308

In [31]:
opt_q4.x

array([ 88.39998865,  93.71901398,  96.62338732,  86.96668235,
        92.63749163,  83.68963504,  91.4218463 ,  80.76474943,
        95.99872814,  89.51351498,  74.38166998,  96.13316597,
        90.86215314,  94.74019116,  82.31758103,  80.76596104,
        90.49293898,  95.94242021, 100.        ,  99.50635431,
        88.97647884,  97.02612371,  78.91058884,  94.83576027,
        90.52013549,  71.59766645,  96.4835024 ,  89.11958048,
        78.60943489,  83.04860283,  86.06605345,  84.01395421,
         2.17271501])

In [32]:
def correct_prediction(actual, predicted):
    if actual*predicted > 0:
        return 1
    else:
        return 0

In [33]:
# getting predicted spread
_, predicted_spread = prediction_error(opt_q4.x)
rating['predicted_spread'] = predicted_spread
rating['correct_pred'] = rating.apply(lambda row: correct_prediction(row['actual_spread'], row['predicted_spread']), axis=1)
rating

,week,home_team_index,visit_team_index,home_score,visit_score,actual_spread,predicted_spread,correct_pred
0,1,25,31,13,10,3,6.626797,1
1,1,2,17,19,7,12,5.398790,1
2,1,29,26,28,0,28,9.184483,1
3,1,21,32,23,17,6,7.135240,1
4,1,3,16,38,24,14,18.030141,1
...,...,...,...,...,...,...,...,...
251,17,5,20,23,10,13,-4.696148,0
252,17,11,6,23,37,-14,-7.135250,1
253,17,22,6,37,0,37,15.509204,1
254,17,8,15,23,17,6,0.619883,1


In [34]:
rating['correct_pred'].sum()

181